In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
from numpy.ma.core import less_equal
from tqdm import tqdm

In [2]:
# Create Model
m = gp.Model("Model_1")

Set parameter Username
Set parameter LicenseID to value 2595554
Academic license - for non-commercial use only - expires 2025-12-04


In [3]:
# GENERAL PARAMETERS

# Number of vehicle types
V = 6

# Number of nodes i,j
nodes = 4

# I, J = nodes, nodes
arcs = {i: [j for j in range(nodes) if abs(i - j) <= 1] for i in range(nodes)}

# Time Steps (days)
T = 11

# Commodity variable types
X = [GRB.INTEGER, GRB.CONTINUOUS, GRB.CONTINUOUS, GRB.CONTINUOUS, GRB.CONTINUOUS]
# Crew, consumables, equipment, samples, propellant

Y = GRB.INTEGER
# Spacecrafts of same type

In [4]:
# ASSUMPTIONS

# Consumption rates [kg/crew/day]
food_consumption = 1.015
water_consumption = 6.37
oxygen_consumption = 1.18
consumption = food_consumption + water_consumption + oxygen_consumption

# Crew mass [kg/crew]
crew_mass = 100

# Gravitational acceleration [m/sˆ2]
g_0 = 9.8


In [5]:
# VEHICLE DATA

# Structure mass [kg]
s = np.array([38415, 12014, 4841, 6053, 2770, 1719])

# Specific impulses [s]
I_sp = np.array([421, 421, 0, 314, 311, 311])

# Payload Capacity [kg]
C = np.array([0, 0, 524, 60, 500, 250])

# Propellant Capacity [kg]
M = np.array([452045, 107725, 0, 18413, 8804, 2358])


In [6]:
# DISPLACEMENT DATA

# Velocity change [km/s]
# PAC, LEO, LLO, LS are 0, 1, 2, 3
delta_V = {0: {0: 0, 1: 0}, 1: {0: 0, 1: 0, 2: 4.04}, 2: {1: 4.04, 2: 0, 3: 1.87}, 3: {2: 1.87, 3: 0}}

# Time of fare [days]
TOF = {0: {0: 1, 1: 1}, 1: {0: 1, 1: 1, 2: 3}, 2: {1: 3, 2: 1, 3: 1}, 3: {2: 1, 3: 1}}

# Propellant mass fraction
phi = [[{j: 1 - np.exp(-(delta_V[i][j] / (I_sp[v] * g_0))) if I_sp[v] != 0 else 1 for j in arcs[i]}
        for i in arcs]
       for v in range(V)]


In [7]:
# CREATE COMMODITY FLOW VECTORS AND S/C COMMODITY FLOW

def create_commodity_flow(model, V, arcs, T, X, direction):
    x_flow = [[{j: [np.array([[model.addVar(vtype=X[x], name=f'commodity_{direction}flow_{v},{i},{j},{t},{x}')]
                              for x in range(len(X))])
                    for t in range(T-1)]
                for j in arcs[i]}
               for i in arcs]
              for v in range(V)]

    return x_flow


def create_sc_commodity_flow(model, V, arcs, T, Y, direction):
    y_flow = [
        [{j: [np.array([model.addVar(vtype=Y, name=f'sc_commodity_{direction}flow_{v},{i},{j},{t}')]) for t in range(T-1)]
          for j in arcs[i]}
         for i in arcs]
        for v in range(V)]

    return y_flow

# Outflow+ leaving from node i to j, inflow- arriving at node j from i

x_outflow, x_inflow = create_commodity_flow(m, V, arcs, T, X, "out"), create_commodity_flow(m, V, arcs, T, X, "in")
y_outflow, y_inflow = create_sc_commodity_flow(m, V, arcs, T, Y, "out"), create_sc_commodity_flow(m, V, arcs, T, Y, "in")

m.update()


In [ ]:
# # Create commodity flow variables and add them to model
# def create_add_commodity_flow(model, V=V, I=I, J=J, T=T):
#     x_outflow, x_inflow = {}, {}
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[0, v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name='commodity_outflow_crew')  # Different names??
#         x_inflow[0, v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name='commodity_inflow_crew')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[1, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_equipment')
#         x_inflow[1, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_equipment')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[2, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_samples')
#         x_inflow[2, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_samples')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[3, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_consumables')
#         x_inflow[3, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_consumables')
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         x_outflow[4, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_outflow_propellant')
#         x_inflow[4, v, i, j, t] = model.addVar(vtype=GRB.CONTINUOUS, name='commodity_inflow_propellant')
#     return x_outflow, x_inflow
#
#
# x_outflow, x_inflow = create_add_commodity_flow(model=m)
#
#
# # Create number of spacecraft flow variables and add them to model
# def create_add_number_spacecraft_per_arc(model, V=V, I=I, J=J, T=T):
#     y_outflow, y_inflow = {}, {}
#     for v, i, j, t in tqdm([(v, i, j, t) for v in range(V) for i in range(I) for j in range(J) for t in range(T)]):
#         y_outflow[v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name=f"spacecraft_ouflow_{v}{i}{j}{t}")
#         y_inflow[v, i, j, t] = model.addVar(vtype=GRB.INTEGER, name=f"spacecraft_inflow_{v}{i}{j}{t}")
#     return y_outflow, y_inflow
#
#
# y_outflow, y_inflow = create_add_number_spacecraft_per_arc(model=m)
#
# m.update()  #??

In [8]:
# CONSTRAINTS 2 & 3 MASS BALANCE
# Node commodity demand D vectors (positive for supply)
# sum(x[i][t]+) - sum(x[i][t]-) <= D[i][t]
# x = Crew, consumables, equipment, samples, propellant

# COMMODITY DEMAND
D = [[np.array([999999 if ((x != 3 and i == 1) or (x == 3 and i == 3)) else 0 for x in range(len(X))])
      for _ in range(T)]
    for i in arcs]

# Earth (LEO) crew, consumables, equipment, propellant supply, AND Moon surface sample supply (infinite)
# THE COMMODITIES ARE PROVIDED AT LEO ????
# CHECK DAYS FOR MISSION !!!

# APOLLO

# for t in range(T):
#     D[1][t] = np.array([999999 if x != 3 else 0 for x in range(len(X))])
#     D[3][t][3] = 999999

# Crew demand/supply
D[3][4][0] = -2 # Lunar surface day 5 crew demand (negative supply)
D[2][3][0] = -1 # Lunar orbit day 4 crew demand
D[3][5][0] = 2 # Lunar surface day 6 crew supply (return)
D[2][6][0] = 1 # Lunar orbit day 7 crew supply (return)
D[0][10][0] = -3 # Earth day 11 crew demand (return)

D[3][4][2] = -420 # Lunar surface day 5 (scientific) equipment demand

D[0][10][3] = -110 # Earth day 11 lunar sample demand


# S/C COMMODITY DEMAND
d = [[[0 if i != 1 else 999999 for _ in range(T)] # Infinite supply of spacecrafts at LEO ???
     for _ in range(V)]
    for i in arcs]


In [9]:
# ADD THE CONSTRAINTS (2 & 3)

for i in arcs:
    for t in range(T):

        x_outflow_sum = sum(x_outflow[v][i][j][t] for v in range(V) for j in arcs[i]) if t < T-1 \
            else np.array([[0] for _ in range(len(X))])
        # On the last day there is no outflow

        x_inflow_sum = sum(x_inflow[v][j][i][t - TOF[j][i]] if t >= TOF[j][i]
                           else np.array([[0] for _ in range(len(X))])
                           for v in range(V) for j in arcs[i])
        # Only count the inflows for which the spacecraft has had time to arrive

        for x in range(len(X)):
            m.addConstr(x_outflow_sum[x][0] - x_inflow_sum[x][0] <= D[i][t][x])

        # S/C commodity supply and demand
        for v in range(V):
            y_outflow_sum = sum(y_outflow[v][i][j][t] for j in arcs[i]) if t < T-1 \
                else np.array([0])

            y_inflow_sum = sum(y_inflow[v][j][i][t - TOF[j][i]] if t >= TOF[j][i]
                               else np.array([0])
                               for j in arcs[i])

            m.addConstr(y_outflow_sum[0] - y_inflow_sum[0] <= d[i][v][t])

m.update()

In [10]:
# CONSTRAINTS 4 COMMODITY TRANSFORMATION

# Commodity transformation matrix
# Q[x+, y+] = [x-, y-] --> Difference between what leaves from node i and what arrives at node j. Ex. propellant use
# x = Crew, consumables, equipment, samples, propellant

def create_commodity_transformation(V, arcs, consumption, crew_mass, phi, TOF):
    Q = [[{j: np.array([[1, 0, 0, 0, 0, 0],
                        [-consumption * TOF[i][j], 1, 0, 0, 0, 0], # Consumable consumption
                        [0, 0, 1, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0],
                        [crew_mass * -phi[v][i][j], -phi[v][i][j], -phi[v][i][j], -phi[v][i][j], 1 - phi[v][i][j], -phi[v][i][j]], # Propellant consumption
                        [0, 0, 0, 0, 0, 1]])
           for j in arcs[i]}
          for i in arcs]
         for v in range(V)]

    return Q

Q = create_commodity_transformation(V, arcs, consumption, crew_mass, phi, TOF)

In [11]:
# ADD THE CONSTRAINTS (4)

for v in range(V):
    for i in arcs:
        for j in arcs[i]:
            for t in range(T-1):
                for leaving, arriving in zip(np.dot(Q[v][i][j], np.concatenate((x_outflow[v][i][j][t],
                                                                                np.array([s[v]*y_outflow[v][i][j][t]])), axis=0)),
                                             np.concatenate((x_inflow[v][i][j][t], np.array([s[v]*y_inflow[v][i][j][t]])), axis=0)):

                    m.addConstr(leaving[0] == arriving[0])


# m.addConstr(Q[v][i][j] * np.concatenate((x_outflow[v][i][j][t], np.array([s[v]*y_outflow[v][i][j][t]])), axis=0) ==
#                             np.concatenate((x_inflow[v][i][j][t], np.array([s[v]*y_inflow[v][i][j][t]])), axis=0))

m.update()


In [12]:
# CONSTRAINTS 5 CONCURRENCY LIMITS

# Concurrency constraint matrix
# H[x+] <= e * y+ --> Payload mass and fuel in s/c does not exceed maximum capacities
# x = Crew, consumables, equipment, samples, propellant

# def create_concurrency_constraint(V, arcs, crew_mass): # Different vehicles version
#     H = [[{j: np.array([[crew_mass, 1, 1, 1, 0],
#                         [0, 0, 0, 0, 1]])
#            for j in arcs[i]}
#           for i in arcs]
#          for v in range(V)]
#
#     return H


def create_concurrency_constraint(arcs, crew_mass): # Same for all vehicles
    H = [{j: np.array([[crew_mass, 1, 1, 1, 0],
                        [0, 0, 0, 0, 1]])
           for j in arcs[i]}
          for i in arcs]
    return H


def create_sc_design_parameters(V, C, M):
    e = [np.array([[C[v]], [M[v]]]) for v in range(V)]
    return e


H = create_concurrency_constraint(arcs, crew_mass)
e = create_sc_design_parameters(V, C, M)

In [13]:
# ADD THE CONSTRAINTS (5)

for v in range(V):
    for i in arcs:
        for j in arcs[i]:
            for t in range(T-1):
                for commodity, constraint in zip(np.dot(H[i][j], x_outflow[v][i][j][t]),
                                                 e[v]*y_outflow[v][i][j][t][0]):

                    m.addConstr(commodity[0] <= constraint[0])

m.update()


In [14]:
# CONSTRAINTS 6 TIME-WINDOW
# ADD THE CONSTRAINTS (6)

for v in range(V):
    for i in arcs:
        for j in arcs[i]:
            for t in range(T-1):
                for commodity_out in x_outflow[v][i][j][t]:
                    m.addConstr(commodity_out[0] >= 0)

                for commodity_in in x_inflow[v][i][j][t]:
                    m.addConstr(commodity_in[0] >= 0)

                m.addConstr(y_outflow[v][i][j][t][0] >= 0)
                m.addConstr(y_inflow[v][i][j][t][0] >= 0)

m.update()

# s[v] >= 0


In [15]:
# CONSTRAINTS 7 SPACE-CRAFT MASS

In [16]:
# CONSTRAINT 1 COST FUNCTION - INITIAL MASS AT LEO
# sum(cost * x + cost_y * s * y)

# x = Crew, consumables, equipment, samples, propellant

def create_commodity_cost(V, arcs, crew_mass):
    cost_coeff = [[{j:
                        [np.array([[crew_mass], [1], [1], [1], [1]]) if (t == 0 and i == 1)
                         else np.array([[0] for _ in range(len(X))])
                         for t in range(T-1)]
           for j in arcs[i]}
          for i in arcs]
         for _ in range(V)]

    sc_cost_coeff = [[{j:
                        [1 if (t == 0 and i == 1)
                         else 0
                         for t in range(T-1)]
           for j in arcs[i]}
          for i in arcs]
         for _ in range(V)]

    return cost_coeff, sc_cost_coeff

cost_coeff, sc_cost_coeff = create_commodity_cost(V, arcs, crew_mass)

In [17]:
# DEFINE THE COST FUNCTION (1)

cost = sum(
    np.dot(cost_coeff[v][i][j][t].T, x_outflow[v][i][j][t]) + sc_cost_coeff[v][i][j][t] * s[v] * y_outflow[v][i][j][t][0]
    for v in range(V)
    for i in arcs
    for j in arcs[i]
    for t in range(T-1)
)

cost = cost[0][0]

m.setObjective(cost, GRB.MINIMIZE)
m.update()

In [18]:
m.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 23.6.0 23G80)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 12484 rows, 7200 columns and 27356 nonzeros
Model fingerprint: 0x048a46d5
Variable types: 4800 continuous, 2400 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-04, 5e+05]
  Objective range  [1e+00, 4e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+06]
Presolve removed 12226 rows and 6799 columns
Presolve time: 0.02s
Presolved: 258 rows, 401 columns, 1365 nonzeros
Variable types: 258 continuous, 143 integer (1 binary)

Root relaxation: objective 5.334590e+03, 68 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 5334.59004    0   10          - 5334.59004      -     -    0s
H    0     0         

In [20]:
for final_variable in m.getVars():
    if final_variable.X != 0:
        print('%s %g' % (final_variable.VarName, final_variable.X))

commodity_outflow_0,1,1,2,4 999999
commodity_outflow_0,1,2,1,4 0.000147231
commodity_outflow_0,2,3,4,4 1.76846
commodity_outflow_3,1,0,9,0 3
commodity_outflow_3,1,0,9,1 25.695
commodity_outflow_3,1,0,9,3 110
commodity_outflow_3,1,2,1,4 994301
commodity_outflow_3,2,1,5,4 429.413
commodity_outflow_4,1,0,9,0 3
commodity_outflow_4,1,0,9,1 25.695
commodity_outflow_4,1,2,0,0 3
commodity_outflow_4,1,2,0,1 94.215
commodity_outflow_4,1,2,0,2 420
commodity_outflow_4,1,2,0,4 12.2246
commodity_outflow_4,2,3,3,0 2
commodity_outflow_4,2,3,3,1 17.13
commodity_outflow_4,2,3,3,2 420
commodity_outflow_4,2,3,3,4 3.79119
commodity_outflow_4,3,2,5,3 110
commodity_outflow_4,3,2,5,4 1.76759
commodity_outflow_5,1,0,9,0 3
commodity_outflow_5,1,0,9,1 25.695
commodity_outflow_5,1,2,1,4 5698.03
commodity_outflow_5,2,1,6,3 110
commodity_outflow_5,2,1,6,4 2.42603
commodity_outflow_5,2,2,4,4 431.839
commodity_outflow_5,2,2,5,4 2.42603
commodity_inflow_0,1,1,2,4 999999
commodity_inflow_0,2,3,4,4 1.76759
commodity_inf

In [ ]:
# # EQUATION 7 CONSTRAINTS
#
# # Structural Fraction (fuel dependent)
# alpha = 0.045  # LOX/kerosene
#
# # Gravitational Acceleration Earth
# g_0 = 9.8  # m/s2
#
# # Upper Bound Allowed for Propellant Tank Capacity
# M_ub = 500000  # kg
#
# # Spacecraft Impulsive Burn
# t_b = 120  # s
#
#
# # Structure Mass Variable
# def create_s_star_variables(model, v=V):
#     variables = {}
#     for v in range(V):
#         variables[v] = model.addVar(vtype=GRB.CONTINUOUS, name=f'Structure_Mass_{v}')
#     return variables
#
#
# s_star = create_s_star_variables(model=m)
#
# m.update()

In [ ]:
# # CONSTRAINTS 7
#
# for v in tqdm(V):
#     m.addConstr(s_star[v] = 2.3931 * )